In [54]:
def test_sym(A_val, A_rowval, colptr):
    n=len(colptr)-1
    A_colptr=colptr.copy() #make a copy so we dont destroy input data
    #take O(n) storage of integers
    for i in range(n-1):
        #This part helps us skip the diagonal entries
        if (A_rowval[A_colptr[i]]==i): #1 branching check if it is an entry of form (i,i)
            A_colptr[i]+=1 #1 reading+1 writing
        #Overal we have n-1 branching and O(n) reading+writing here
        for j in range(A_colptr[i],colptr[i+1]): 
            #this loop(the value of A_val[j]) (essentially) only goes through the non-zero entries in the strictly lower triangular part
            #because we skip the diagonal entries and we "delete" the nz entries in the i-th row 
            #corresponding to the nz entries of the i-th column after i-th iteration
            #Note it is possible for the loop (A_val[j]) to land on an upper triangular entry, but in this case, it means we did not "delete"
            #this entry in previous iterations, meaning the corresponding lower triangular entry must be zero and the loop
            #will immediately terminate and return false
            #Also note this loop can repeat at most ceil(N/2) times because each time we visit one non-zero entry in lower triangular part and 
            #its counterpart in the upper triangular part, and each non-zero entry can only be visited once by construction (thanks to the deletion operation)
            #Therefore, after ceil(N/2) iterations, it would have visited all the nonzero entries and must stop
            #Thus total number of iterations for j is ceil(N/2)
            row_ind=A_rowval[j]
            colp=A_colptr[row_ind]
            row_ind2=A_rowval[colp]
            # 3 reading+writing, note the storage in the loop are temporary and reused, so they only take O(1) space
            if ((i!=row_ind2) or (A_val[j]!=A_val[colp])): #check corresponding upper triangular entry is non-zero and has same value
                # 2 readings, 3 boolean operations and 1 branching
                return False
            A_colptr[row_ind]+=1 #1 reading, 1 writing
            #this essentially "deletes" the corresponding nz upper triangular entries in the i-th row
            #since at this point we have checked that i-th col is the same as the i-th row
        #Thus for each j, we perform O(1) reading and writing, O(1) boolean operations and 1 branching
        #Total number of instructions is O(N) and total number of branching is at most ceil(N/2)
    #Combining everything we see that we use O(n) extra storage of integers
    #O(n+N) instructions
    #1+n-1+N/2=n+N/2
    return True

In [55]:
def run_sym_test():
    import scipy.sparse
    import numpy as np
    import random
    n = 64
    for k in range(1):
        A = np.zeros([n,n])
        p = random.random()
        nnz = 0
        for i in range(n):
            for j in range(n):
                if (random.random() > p):
                    A[i, j] = i
                    if (i!=j):
                        nnz = nnz + 1
        A_sparse=scipy.sparse.csc_matrix(A)
        Ap=A_sparse.data.tolist()
        Ai=A_sparse.indices.tolist()
        Ax=A_sparse.indptr.tolist()
        if (nnz == 0):
        # if nnz = 0 the matrix is symmetric
            assert(test_sym (Ap , Ai , Ax))
        else:
            assert(~test_sym (Ap , Ai , Ax))
        A=A+np.transpose(A)
        A_sparse=scipy.sparse.csc_matrix(A)
        Ap=A_sparse.data
        Ai=A_sparse.indices
        Ax=A_sparse.indptr
        assert(test_sym (Ap , Ai , Ax))
    print("Test Successful!")      

In [56]:
run_sym_test()

Test Successful!
